### Bokeh_Plots_Site.ipynb

Brian Dew, @bd_econ, April 22, 2018

----

Description: Retrieve GDP data and make a bar plot showing GDP growth.

In [1]:
#Import preliminaries
import requests
import pandas as pd
from datetime import datetime as dt
import time
import json
import bls

# Local file with API key
import config
api_key = config.bea_key

# Bokeh settings
from bokeh.plotting import figure, show
import bokeh.models
import bokeh.plotting
from bokeh.models import ColumnDataSource, Span, Label, HoverTool, Range1d, BoxAnnotation, DatetimeTickFormatter
from bokeh.embed import components
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# US GDP

In [2]:
# Individual parameters of the request
uri = f'https://www.bea.gov/api/data/?&UserID={api_key}'
mdt = '&method=GetData&DataSetName=NIPA&TableName=T10101'
yr = ','.join(map(str, range(2010, 2019)))

# Construct URL from parameters above
url = f'{uri}{mdt}&Frequency=Q&Year={yr}&ResultFormat=json'

# Request parameter information from BEA API
r = requests.get(url).json()['BEAAPI']['Results']

# GDP data as a pandas dataframe
df = pd.DataFrame(
    [(pd.to_datetime(i['TimePeriod']), float(i['DataValue'])) 
     for i in r['Data'] if i['SeriesCode'] == 'A191RL'], 
    columns=['Date', 'Value']).set_index('Date')

# String dates
df['strdate'] = [f'{i.year} Q{i.quarter}' for i in df.index]

In [3]:
# GDP plot in Bokeh
source = ColumnDataSource({'x': df.index, 
                           'y': df['Value'], 
                           'Date': df['strdate']})

tooltips = """
    <div>
        <span style="font-size: 12px;">@Date:</span>
        <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 800.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

p = figure(plot_width=720, plot_height=400, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-2.6, 6.2))
p.vbar('x', width=width, bottom=0, top='y', 
       source=source, color='Red', alpha=1.0, 
       level='overlay')
p.add_layout(zero_line)
p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.toolbar.active_drag = None
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False))

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/gdp.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

### Growth by component

In [7]:
df = pd.read_csv('C:/Working/Python/Macro_Dash/data/gdp_comp2.csv', parse_dates=['date'], index_col='date')

In [8]:
df

,A006RY,A019RY,A191RL,A822RY,DPCERY
date,,,,,
2006-01-01,1.25,0.23,5.4,0.96,2.98
2006-04-01,-0.51,0.13,0.9,-0.03,1.34
2006-07-01,-0.33,-0.58,0.6,-0.11,1.63
2006-10-01,-1.55,1.73,3.5,0.64,2.63
2007-01-01,-0.58,-0.14,0.9,0.13,1.54
2007-04-01,0.80,0.28,2.3,0.71,0.52
2007-07-01,-0.77,1.08,2.2,0.35,1.52
2007-10-01,-0.89,1.99,2.5,0.60,0.76
2008-01-01,-1.87,0.17,-2.3,0.17,-0.74


In [9]:
# Clean up data 
df.columns = ['GPDIC1', 'NETEXC', 'GDPC1', 'GCEC1', 'PCECC96']
df['date'] = df.index
# String dates
df['strdate'] = [f'{i.year} Q{i.quarter}' for i in df.index]
df['date2'] = df.index.to_period("Q").astype('str')
# Column names
col_names = ['GPDIC1', 'NETEXC', 'GCEC1', 'PCECC96']

# Split positive and negative values
gr = {i :[n if n > -0.0001 else 0 for n in list(df[i])] for i in col_names}
gr['dates'] = list(df.index)
gr['strdate'] = df.index.strftime('%Y-Q%q')
sl = {i :[n  if n < -0.0001 else 0 for n in list(df[i])] for i in col_names}
sl['dates'] = list(df.index)
sl['strdate'] = df.index.strftime('%Y-Q%q')

In [10]:
# HTML text that shows when hovering
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 15px; font-style: italic;">@strdate:</span><br>
        <span style="font-size: 10px; padding: 0 5px;">GDP Growth: </span>
        <span style="font-size: 11px; font-weight: bold;">@GDPC1{1.1}%</span><br>
        <span style="font-size: 9px; font-style: italic;">Contribution from:</span><br>
        <div style="text-align: right;">
            <span style="font-size: 10px; padding: 0 5px;">Government: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@{GCEC1}{1.1}</span>
            <span style="display: inline-block; color: #0061c8; width: 8px;">▮</span><br> 
            <span style="font-size: 10px; padding: 0 5px;">Investment: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@{GPDIC1}{1.1}</span>
            <span style="display: inline-block; color: #ffe827; width: 8px;">▮</span><br>
            <span style="font-size: 10px; padding: 0 5px;">Net exports: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@NETEXC{1.1}</span>
            <span style="display: inline-block; color: #80DEEA; width: 8px;">▮</span><br>
            <span style="font-size: 10px; padding: 0 5px;">Consumption: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@{PCECC96}{1.1}</span>
            <span style="display: inline-block; color: #00a443; width: 8px;">▮</span><br>
        </div>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 750.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

p = figure(plot_width=720, plot_height=400, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-10.0, 8.0),
           x_range=Range1d(pd.to_datetime('2005-12-01'), pd.to_datetime('2019-02-01')))

p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))

p.vbar_stack(col_names, x='dates', width=width, level='overlay', fill_alpha=1.0,
             color=['#ffe827','#80DEEA' ,'#0061c8', '#00a443'],
             legend=['Investment', 'Net exports', 'Government', 'Consumption'],
             line_alpha=0, source=ColumnDataSource(gr))
p.vbar_stack(col_names, x='dates', width=width, level='overlay', fill_alpha=1.0,
             color=['#ffe827','#80DEEA' ,'#0061c8', '#00a443'], 
             line_alpha=0, source=ColumnDataSource(sl))

p.add_layout(zero_line)

r1 = p.line('date', 'GDPC1', color='black', level='overlay', line_width=1.5, alpha=1.0, source=ColumnDataSource(df))

p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None

p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.toolbar.active_drag = None
p.legend.orientation = "horizontal"
p.legend.location = "top_left"
p.legend.border_line_alpha = 0
p.legend.background_fill_alpha = 0
p.legend.margin = 2
p.legend.padding = 0
p.legend.spacing = 5
p.legend.label_standoff = 1
p.legend.glyph_height = 16
p.legend.glyph_width = 8
p.legend.label_text_font_size = '8.5pt'

p.circle(x=df.index[-1], y=df.GDPC1[-1], size=5, color='black', name='latest', level='overlay')
p.text(x=df.index[-1], y=df.GDPC1[-1]+0.6, text=['2018\nQ2:\n4.1%'], x_offset=-4,
       text_font_size='8pt', text_align='left', text_baseline='bottom')

p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False, mode='vline',  renderers=[r1]))

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/gdp_comp.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

### Labor Productivity

In [2]:
df = bls.api({'PRS85006092': 'PRS85006092'}, (2000, 2018))

Post Request Status: REQUEST_SUCCEEDED


In [3]:
#df = pd.read_csv('PRS85006092.csv', parse_dates=['DATE'], index_col='DATE').loc['2000':]
df['strdate'] = [f'{i.year} Q{i.quarter}' for i in df.index]

In [4]:
str_date = df['strdate'].iloc[-1].replace(' ', '\n')
str_val = f"{df['PRS85006092'].iloc[-1]}%"

In [6]:
# GDP plot in Bokeh
source = ColumnDataSource({'x': df.index, 
                           'y': df['PRS85006092'], 
                           'Date': df['strdate']})

tooltips = """
    <div>
        <span style="font-size: 12px;">@Date:</span>
        <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 800.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

p = figure(plot_width=720, plot_height=400, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-6.0, 12.0),
           x_range=Range1d(pd.to_datetime('1999-12-01'), pd.to_datetime('2019-06-01')))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2001-04-01'), right=pd.to_datetime('2001-12-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.vbar('x', width=width, bottom=0, top='y', 
       source=source, color='Red', alpha=1.0, 
       level='overlay')
p.add_layout(zero_line)
p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.toolbar.active_drag = None
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False))
p.text(x=df.index[-1], y=3.1, text=[f'{str_date}:\n{str_val}'], x_offset=-4,
       text_font_size='8pt', text_align='left', text_baseline='bottom')

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/labor_productivity.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

### CPI Long-term

In [2]:
df = pd.read_csv('C:/Working/USA/datafiles/cpi.csv', parse_dates=['date'], index_col='date')[['value', 'value2']]

In [3]:
str_date = f'{df.index[-1]:%Y-%m}'
str_val = f"{df['value'].iloc[-1]:2.1f}%"
str_val2 = f"{df['value2'].iloc[-1]:2.1f}%"

In [4]:
#Plotting CPI
source = ColumnDataSource({'x': df.index, 'yall': df.value, 'ycore': df.value2, 'Date': df.index.strftime('%b %Y')})
tooltipsde = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">All-items</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@yall{1.1}%</span>
            </div>
"""
tooltipsuk = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">Core</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@ycore{1.1}%</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=0.8)
p = figure(width=720,height=400,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None,
           toolbar_location=None, toolbar_sticky=False, 
           y_range=Range1d(-3.0, 6.5), 
           x_range=Range1d(pd.to_datetime('2005-12-01'), pd.to_datetime('2019-09-01')))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
r1 = p.line('x', 'yall', source=source,  
            line_width=3.0, color='#09a8fa', alpha=1.0, level='overlay')
r2 = p.line('x', 'ycore', source=source,
            line_width=1.5, color='gray', alpha=1.0, level='overlay')
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltipsde, renderers=[r1], show_arrow=False))
p.add_tools(HoverTool(tooltips=tooltipsuk, renderers=[r2], show_arrow=False))
p.toolbar.active_drag = None
p.text(x=pd.to_datetime('2011-03-01'), y=0.9, text=['Core'], 
       text_font_style='bold', text_font_size='10pt', 
       text_align='left', text_color='gray', text_baseline='top')
p.circle(x=df.index[-1], y=df['value2'][-1], size=5, color='gray')
p.text(x=df.index[-1], y=df['value2'][-1] - 0.2, text=[f'{str_val2}'], 
       x_offset=32, text_font_size='8pt', text_align='right', 
       text_baseline='bottom')
p.text(x=pd.to_datetime('2012-02-01'), y=3.8, text=['All-items'], 
       text_font_style='bold', text_font_size='10pt', text_align='left', 
       text_color='#09a8fa', text_baseline='top')
p.circle(x=df.index[-1], y=df['value'][-1], size=6, color='#09a8fa')
p.text(x=df.index[-1], y=df['value'][-1] -0.05, 
       text=[f'{str_date}\n{str_val}'], x_offset=32, text_font_size='8pt', 
       text_align='right', text_baseline='bottom')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/cpi_lt.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

### Trade Balance

In [2]:
# Trade balance
df = pd.read_csv('trade_balance.csv', parse_dates=['DATE'], index_col='DATE')

source = ColumnDataSource({'x': df.index, 'y': df.values, 'Date': [f'{i.year} Q{i.quarter}' for i in df.index]})

tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
            </div>
"""
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)

p = figure(width=720,height=400,x_axis_type='datetime', tools=['pan, wheel_zoom,box_zoom,reset'],
           logo=None, toolbar_location=None, toolbar_sticky=False, y_range=Range1d(-6.4, 0.1), 
           x_range=Range1d(pd.to_datetime('1991-12-01'), pd.to_datetime('2019-12-01')))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.add_layout(BoxAnnotation(left=pd.to_datetime('2001-04-01'), right=pd.to_datetime('2001-12-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.line('x', 'y', source=source, line_width=3, color='#2e86c1', name='line')
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
#p.grid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltips, names=['line'], show_arrow=False, point_policy='snap_to_data'))
p.toolbar.active_drag = None
p.circle(x=df.index[-1], y=df.values[-1], size=6, color='#2e86c1', name='latest')
p.text(x=df.index[-1], y=df.values[-1]+0.2, text=['2018 Q1     \n-3.3%'], text_font_size='8pt', x_offset=14,
       text_align='center', text_baseline='bottom')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/trade_balance.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

### USD index

In [3]:
# The URL for the FRED API
base = 'https://api.stlouisfed.org/fred/series/observations?series_id='
# List of FRED series IDs and their description
s_dict = {'TWEXB': 'Trade-weighted dollar'}

# Include start date, API key from config.py file and file type json
start_date = '1994-01-01'
dates = '&observation_start={}'.format(start_date)
api_key = '&api_key={}'.format(config.fred_key)
ftype = '&file_type=json'
df = pd.DataFrame()
for code, name in s_dict.items():
    url = '{}{}{}{}{}'.format(base, code, dates, api_key, ftype)
    r = requests.get(url).json()['observations']
    df[name] = [float(i['value']) for i in r]
df.index = pd.to_datetime([i['date'] for i in r])

In [4]:
str_date = f'{df.index[-1]:%b %d}'
str_val = f"{df['Trade-weighted dollar'].iloc[-1]:4.2f}"

In [5]:
# USD index
source = ColumnDataSource({'x': df.index, 'y': df.values, 'Date': df.index.format()})

tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.111}</span>
            </div>
"""
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)

p = figure(width=720,height=400,x_axis_type='datetime', tools=['pan, wheel_zoom,box_zoom,reset'],
           logo=None, toolbar_location=None, toolbar_sticky=False, y_range=Range1d(85, 135), 
           x_range=Range1d(pd.to_datetime('1994-12-01'), pd.to_datetime('2020-02-01')))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.add_layout(BoxAnnotation(left=pd.to_datetime('2001-04-01'), right=pd.to_datetime('2001-12-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.line('x', 'y', source=source, line_width=2, color='#03bf00', name='line')
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
#p.grid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltips, names=['line'], show_arrow=False, point_policy='snap_to_data'))
p.toolbar.active_drag = None
p.circle(x=df.index[-1], y=df.values[-1], size=5, color='#03bf00', name='latest')
p.text(x=df.index[-10], y=df.values[-1]+1.2, text=[f'{str_date}\n{str_val}'], text_font_size='8pt', x_offset=9,
       text_align='center', text_baseline='bottom')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/USD_index.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

### Treasury Yield - 2 and 10 year bonds

In [6]:
df = pd.read_csv('C:/Working/USA/datafiles/twotentry.csv', parse_dates=['date'], 
                 index_col='date')[['twoyr', 'tenyr']]
str_date = f'{df.index[-1]:%b %d}'

In [7]:
#Plotting treasury yields
source = ColumnDataSource({'x': df.index, 'y2': df.twoyr, 'y10': df.tenyr, 'Date': df.index.format()})
tooltipsde = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">Two-year bond yield</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y2{1.11}%</span>
            </div>
"""
tooltipsuk = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">Ten-year bond yield</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y10{1.11}%</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=0.8)
p = figure(width=720,height=400,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None,
           toolbar_location=None, toolbar_sticky=False, 
           y_range=Range1d(0.0, 5.8), 
           x_range=Range1d(pd.to_datetime('2005-12-01'), pd.to_datetime('2019-09-01')))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
r1 = p.line('x', 'y2', source=source,  
            line_width=1.5, color='#ffc300', alpha=1.0, level='overlay')
r2 = p.line('x', 'y10', source=source,
            line_width=1.5, color='#eb0000', alpha=1.0, level='overlay')
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltipsde, renderers=[r1], show_arrow=False))
p.add_tools(HoverTool(tooltips=tooltipsuk, renderers=[r2], show_arrow=False))
p.toolbar.active_drag = None
p.text(x=pd.to_datetime('2011-10-01'), y=3.5, text=['10 year'], text_font_style='bold',
       text_font_size='10pt', text_align='left', text_color='#eb0000', text_baseline='top')
p.circle(x=df.index[-1], y=df['tenyr'][-1], size=5, color='#eb0000')
p.text(x=df.index[-1], y=df['tenyr'][-1]-0.05, text=[f"{str_date}\n{df['tenyr'][-1]:3.2f}"], x_offset=26,
       text_font_size='8pt', text_align='right', text_baseline='bottom')
p.text(x=pd.to_datetime('2011-11-01'), y=0.9, text=['2 year'], text_font_style='bold',
       text_font_size='10pt', text_align='left', text_color='#ffc300', text_baseline='top')
p.circle(x=df.index[-1], y=df['twoyr'][-1], size=5, color='#ffc300')
p.text(x=df.index[-1], y=df['twoyr'][-1] -0.1, text=[f"{df['twoyr'][-1]:3.2f}"], x_offset=26,
       text_font_size='8pt', text_align='right', text_baseline='bottom')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/treasuries.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

### Oil Price (WTI)

In [8]:
df = pd.read_csv('C:/Working/USA/datafiles/wti.csv', parse_dates=['date'], index_col='date')[['value']]

In [9]:
str_date = f'{df.index[-1]:%b %d}'
str_val = f"${df['value'].iloc[-1]:5.2f}"

In [10]:
# WTI oil price
df = pd.read_csv('C:/Working/USA/datafiles/wti.csv', parse_dates=['date'], index_col='date')[['value']]

source = ColumnDataSource({'x': df.index, 'y': df.values, 'Date': df.index.format()})

tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.111}</span>
            </div>
"""
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)

p = figure(width=720,height=400,x_axis_type='datetime', tools=['pan, wheel_zoom,box_zoom,reset'],
           logo=None, toolbar_location=None, toolbar_sticky=False, y_range=Range1d(10, 159), 
           x_range=Range1d(pd.to_datetime('2005-12-01'), pd.to_datetime('2019-12-31')))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.line('x', 'y', source=source, line_width=1.5, color='#ce00a9', name='line')
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
#p.grid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltips, names=['line'], show_arrow=False, point_policy='snap_to_data'))
p.toolbar.active_drag = None
p.circle(x=df.index[-1], y=df.values[-1], size=5, color='#ce00a9', name='latest')
p.text(x=df.index[-4], y=df.values[-1], text=[f'{str_date}\n{str_val}'], text_font_size='8pt', x_offset=20,
       text_align='center', text_baseline='bottom')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/oil_price.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

### Nonfarm payroll growth

In [14]:
df = pd.read_csv('PAYEMS.csv', parse_dates=['DATE'], index_col='DATE').loc['2012':]

In [15]:
if df['PAYEMS_CHG'].iloc[-1] >= 0:
    str_val = f'+{df["PAYEMS_CHG"].iloc[-1]}k'
else:
    str_val = f'-{df["PAYEMS_CHG"].iloc[-1]}k'
    
str_date = f'{df.index[-1]:%Y-%m}'

In [16]:
# GDP plot in Bokeh
source = ColumnDataSource({'x': df.index, 
                           'y': df['PAYEMS_CHG'], 
                           'Date': df.index.format()})

tooltips = """
    <div>
        <span style="font-size: 12px;">@Date:</span>
        <span style="font-size: 12px; font-weight: bold;">@y{int},000</span>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 880.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

p = figure(plot_width=720, plot_height=400, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-160.0, 460.0),
           x_range=Range1d(pd.to_datetime('2011-12-02'), pd.to_datetime('2019-07-01')))
p.vbar('x', width=width, bottom=0, top='y', 
       source=source, color='#5b2c6f', alpha=1.0, 
       level='overlay')
p.add_layout(zero_line)
p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.toolbar.active_drag = None
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False))
p.text(x=df.index[-1], y=250.0, text=[f'{str_date}\n {str_val}'], x_offset=-6,
       text_font_size='8pt', text_align='left', text_baseline='bottom')

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/payrolls.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

### Unemployment decomposition

In [25]:
# Series stored as {id: name}
slist = {'LNS14023621': 'Loser',
         'LNS14023705': 'Leaver',
         'LNS14023557': 'Reentrant',
         'LNS14023569': 'Newentrant',
         'LNS14000000': 'Rate'}

# Start year and end year
dates = ('2012', '2018')

In [26]:
df = bls.api(slist, dates)
df['Date'] = df.index
df['strdate'] = df.index.strftime('%b %Y')
col_names = ['Loser', 'Leaver', 'Reentrant', 'Newentrant']

Post Request Status: REQUEST_SUCCEEDED


In [28]:
# HTML text that shows when hovering
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 15px; font-style: italic;">@strdate:</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Unemployment Rate: </span>
        <span style="font-size: 11px; font-weight: bold;">@Rate{1.1}%</span><br>
        <span style="font-size: 9px; font-style: italic;">Contribution from:</span><br>
        <div style="text-align: right;">
            <span style="font-size: 10px; padding: 0 5px;">New entrants: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@{Newentrant}{1.1}</span>
            <span style="display: inline-block; color: #0022db; width: 8px;">▮</span><br>
            <span style="font-size: 10px; padding: 0 5px;">Re-entrants: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@Reentrant{1.1}</span>
            <span style="display: inline-block; color: #80DEEA; width: 8px;">▮</span><br>
            <span style="font-size: 10px; padding: 0 5px;">Job leavers: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@{Leaver}{1.1}</span>
            <span style="display: inline-block; color: #ff9e34; width: 8px;">▮</span><br>
            <span style="font-size: 10px; padding: 0 5px;">Job losers: </span>
            <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@{Loser}{1.1}</span>
            <span style="display: inline-block; color: #00a125; width: 8px;">▮</span><br> 
        </div>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 750.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

p = figure(plot_width=720, plot_height=400, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-1, 9.9),
           x_range=Range1d(pd.to_datetime('2011-12-01'), pd.to_datetime('2018-09-01')))

p.vbar_stack(col_names, x='Date', width=width, level='overlay', fill_alpha=1.0,
             color=['#00a125', '#ff9e34', '#80DEEA', '#0022db'], 
             legend=['Job losers', 'Job leavers', 'Re-entrants', 'New entrants'], 
             line_alpha=0, source=ColumnDataSource(df))

p.add_layout(zero_line)

r1 = p.line('Date', 'Rate', color='black', level='overlay', line_width=1.5, alpha=0.0, source=ColumnDataSource(df))

p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None

p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.toolbar.active_drag = None
p.legend.orientation = "horizontal"
p.legend.location = "top_left"
p.legend.border_line_alpha = 0
p.legend.background_fill_alpha = 0
p.legend.margin = 2
p.legend.padding = 0
p.legend.spacing = 5
p.legend.label_standoff = 1
p.legend.glyph_height = 16
p.legend.glyph_width = 8
p.legend.label_text_font_size = '8.5pt'

p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False, mode='vline', renderers=[r1]))

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/unemp_decomp.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

### Usual Weekly Earnings

In [24]:
df = pd.read_csv('C:/Working/econ_data/micro/cps_uwe_r.csv', index_col=0, parse_dates=[0])

In [25]:
df.tail()

,25-54,16+
2018-03-01,915.712307,878.222398
2018-04-01,913.882168,877.429222
2018-05-01,910.403236,875.464892
2018-06-01,910.018284,875.622426
2018-07-01,914.899788,877.744451


In [26]:
#Usual weekly earnings by age category
source = ColumnDataSource({'x': df.index, 'y16': df['16+'], 'y25': df['25-54'], 'Date': df.index.strftime('%b %Y')})
tooltipsde = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">Age 16+</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">$@y16{1.11}</span>
            </div>
"""
tooltipsuk = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">Age 25-54</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">$@y25{1.11}</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=720,height=400,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None,
           toolbar_location=None, toolbar_sticky=False, 
           y_range=Range1d(751, 949), x_range=Range1d(pd.to_datetime('1994-12-01'), pd.to_datetime('2020-03-01')))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2001-04-01'), right=pd.to_datetime('2001-12-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
r1 = p.line('x', 'y16', source=source, 
            line_width=2.5, color='#e1315b', alpha=1.0, level='overlay')
r2 = p.line('x', 'y25', source=source,  
            line_width=2.5, color='#008dcb', alpha=1.0, level='overlay')

p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltipsde, renderers=[r1], show_arrow=False))
p.add_tools(HoverTool(tooltips=tooltipsuk, renderers=[r2], show_arrow=False))
p.toolbar.active_drag = None
p.text(x=pd.to_datetime('2004-10-01'), y=825, text=['Age 16+'], text_font_style='bold',
       text_font_size='11pt', text_align='center', text_color='#505050', text_baseline='top')
p.text(x=pd.to_datetime('2004-01-01'), y=915, text=['Age 25-54'], text_font_style='bold',
       text_font_size='11pt', text_align='center', text_color='#505050', text_baseline='bottom')
p.circle(x=df.index[-1], y=df['16+'][-1], size=6, color='#e1315b')
p.text(x=df.index[-1], y=df['16+'][-1] + 2, text=['$878'],
       text_font_size='8pt', text_align='left', text_baseline='bottom')
p.circle(x=df.index[-1], y=df['25-54'][-1], size=6, color='#008dcb')
p.text(x=df.index[-1], y=df['25-54'][-1] + 2, text=['2018-07\n$915'], x_offset=24,
       text_font_size='8pt', text_align='right', text_baseline='bottom')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/cps_uwe.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

# VIX

In [11]:
# VIX retrieved elsewhere, use data since 2015
df = pd.read_csv('C:/Working/USA/datafiles/safe/vix.csv', 
                 parse_dates=['date']).set_index('date')['vixclose']

df = df.loc['2015-01-01':]

In [12]:
str_date = f'{df.index[-1]:%b %d}'
str_val = f'{df.values[-1]:4.2f}'

In [13]:
# VIX plot in bokeh
source = ColumnDataSource({'x': df.index, 'y': df.values, 'Date': df.index.strftime('%b %d, %Y')})
tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=720,height=400,x_axis_type='datetime',
           tools=['pan, wheel_zoom, box_zoom, reset'], logo=None,
           toolbar_location=None, y_range=Range1d(0, 49), 
           x_range=Range1d(pd.to_datetime('2014-12-01'), pd.to_datetime('2019-01-01')))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.line('x', 'y', source = source, line_width=2.5, color='Green', alpha=1.0, level='overlay')
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%b %Y"],
        days=["%b %Y"],
        months=["%b %Y"],
        years=["%b %Y"],
    )
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltips, line_policy='nearest', show_arrow=False))
p.toolbar.active_drag = None
p.text(x=pd.to_datetime('2018-06-30'), y=20, text=[f'{str_date} \n  {str_val}'], text_font_size='8pt', text_align='left', text_baseline='top')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/vix.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

# Germany and UK CPI

In [26]:
# CPI Inflation Data from the IMF
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = f"CompactData/IFS/M.DE+GB+IT+FR.PCPI_PC_CP_A_PT"
data = requests.get(f'{url}{key}').json()

df = pd.DataFrame({s['@REF_AREA'] : {pd.to_datetime(i['@TIME_PERIOD']) : 
     round(float(i['@OBS_VALUE']), 1) 
     for i in s['Obs'] if int(i['@TIME_PERIOD'][:4]) > 1991} 
     for s in data['CompactData']['DataSet']['Series']})

In [27]:
#Plotting CPI for Germany and the UK
source = ColumnDataSource({'x': df.index, 'yde': df.DE, 'yuk': df.GB, 'Date': df.index.strftime('%b %Y')})
tooltipsde = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">Germany</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@yde{1.1}%</span>
            </div>
"""
tooltipsuk = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">United Kingdom</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@yuk{1.1}%</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=720,height=400,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None,
           toolbar_location=None, toolbar_sticky=False)
p.add_layout(hline)
p.sizing_mode = 'scale_width'
r1 = p.line('x', 'yde', source=source, legend='Germany', 
            line_width=2.5, color='#163172', alpha=1.0, level='overlay')
r2 = p.line('x', 'yuk', source=source, legend='United Kingdom', 
            line_width=2.5, color='#09a8fa', alpha=1.0, level='overlay')
p.legend.orientation = "horizontal"
p.legend.location = "top_center"
p.legend.border_line_alpha = 0
p.legend.background_fill_alpha = 0
p.legend.margin = 0
p.legend.padding = 0
p.legend.spacing = 15
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltipsde, renderers=[r1], show_arrow=False))
p.add_tools(HoverTool(tooltips=tooltipsuk, renderers=[r2], show_arrow=False))
p.toolbar.active_drag = None
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/eu_cpi.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

# New Orders

In [14]:
# New orders for capital goods excluding defense or aircraft
key = config.census_key
base = 'https://api.census.gov/data/timeseries/eits/m3'
param = 'cell_value,time_slot_id'
t = '&time=from+2015'
cat = '&category_code=NXA'
dtc = '&data_type_code=MPCNO'
oth = '&for=us&seasonally_adj=yes'

url = f'{base}?get={param}&key={key}{cat}{dtc}{t}{oth}'

r = requests.get(url).json()

df = pd.DataFrame({'date': [pd.to_datetime(i[4]) for i in r[1:]], 
                   'value': [float(i[0]) for i in r[1:]]}).sort_values('date')

In [15]:
# Manually input values from advanced report
df = df.set_index('date')
#df.at[pd.to_datetime('2018-02-01')] = 0.9
#df.at[pd.to_datetime('2018-03-01')] = -0.1

In [16]:
# New Orders plot in Bokeh
source = ColumnDataSource({'x': df.index, 
                           'y': df['value'], 
                           'Date': df.index.strftime('%b %Y')})

tooltips = """
    <div>
        <span style="font-size: 12px;">@Date:</span>
        <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 800.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

# horizontal line at zero
tcja_start = time.mktime(dt(2017, 11, 15, 0, 0, 0).timetuple())*1000
tcja_label = time.mktime(dt(2017, 12, 1, 0, 0, 0).timetuple())*1000
tcja = Span(location=tcja_start, dimension='height', 
                 line_color='gray', line_dash='dashed', line_width=2.0)

p = figure(plot_width=720, plot_height=400, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-4.5, 6.5),
           x_range=Range1d(pd.to_datetime('2014-12-01'), pd.to_datetime('2018-09-01')))
p.vbar('x', width=width, bottom=0, top='y', 
       source=source, color='#ff8d43', alpha=1.0, 
       level='overlay')
#p.add_layout(zero_line)
citation = Label(x=tcja_label, y=4.5, #x_units='screen', y_units='screen',
                 text='TCJA', text_font_size='9pt',#render_mode='css',
                 border_line_color='white', border_line_alpha=0.0,
                 background_fill_color='white', background_fill_alpha=0.0)
p.add_layout(citation)
p.renderers.extend([tcja, zero_line])#p.add_layout(tcja)
p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%b %Y"],
        days=["%b %Y"],
        months=["%b %Y"],
        years=["%b %Y"],
    )
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.toolbar.active_drag = None
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False))

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/neworders.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

# Employed Share of Population

In [27]:
import epop

epop = epop.epop_calc()

In [28]:
epop.keys()

Index(['Total', 'Men', 'Women'], dtype='object')

In [29]:
# Employment rate
source = ColumnDataSource({'x': epop.index, 'y': epop.Total.values, 
                           'men': epop.Men.values, 'women': epop.Women.values, 
                           'Date': epop.index.strftime('%b %Y')})

tooltipsy = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
            </div>
"""
tooltipsmen = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@men{1.1}%</span>
            </div>
"""
tooltipswomen = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@women{1.1}%</span>
            </div>
"""

p = figure(width=720,height=400,x_axis_type='datetime', tools=['pan, wheel_zoom,box_zoom,reset'],
           logo=None, toolbar_location=None, toolbar_sticky=False, y_range=Range1d(66, 91), 
           x_range=Range1d(pd.to_datetime('1994-11-01'), pd.to_datetime('2020-02-01')))
p.sizing_mode = 'scale_width'
p.add_layout(BoxAnnotation(left=pd.to_datetime('2001-04-01'), 
                           right=pd.to_datetime('2001-12-01'), 
                           fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), 
                           right=pd.to_datetime('2009-07-01'), 
                           fill_alpha=0.1, line_alpha=0, fill_color='gray'))
y1 = p.line('x', 'y', source=source, line_width=3, color='Blue', name='line')
m1 = p.line('x', 'men', source=source, line_width=3, color='Green', name='line')
w1 = p.line('x', 'women', source=source, line_width=3, color='Red', name='line')
p.xgrid.grid_line_color = None
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%Y"],
        days=["%Y"],
        months=["%Y"],
        years=["%Y"],
    )
#p.grid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.yaxis.minor_tick_line_color = None
p.add_tools(HoverTool(tooltips=tooltipsy, names=['line'], renderers=[y1],
                      show_arrow=False, point_policy='snap_to_data'))
p.add_tools(HoverTool(tooltips=tooltipsmen, names=['line'], renderers=[m1],
                      show_arrow=False, point_policy='snap_to_data'))
p.add_tools(HoverTool(tooltips=tooltipswomen, names=['line'], renderers=[w1],
                      show_arrow=False, point_policy='snap_to_data'))
p.toolbar.active_drag = None
p.circle(x=epop.index[-1], y=epop.Men.values[-1], size=6, color='Green', name='latest')
p.text(x=epop.index[-1], y=epop.Men.values[-1]+0.4, 
       text=[f'{epop.index[-1]:%b %Y}     \n{epop.Men.iloc[-1]:3.1f}%'], 
       text_font_size='8pt', text_align='center', text_baseline='bottom')
p.circle(x=epop.index[-1], y=epop.Total.values[-1], size=6, color='Blue', name='latest')
p.text(x=epop.index[-1], y=epop.Total.values[-1]+0.4, 
       text=[f'{epop.Total.iloc[-1]:3.1f}%'], 
       text_font_size='8pt', text_align='center', text_baseline='bottom')
p.circle(x=epop.index[-1], y=epop.Women.values[-1], size=6, color='Red', name='latest')
p.text(x=epop.index[-1], y=epop.Women.values[-1]+0.4, 
       text=[f'{epop.Women.iloc[-1]:3.1f}%'], 
       text_font_size='8pt', text_align='center', text_baseline='bottom')
p.text(x=pd.to_datetime('1995-02-01'), y=89.5, text=['Men'], text_font_style='bold',
       text_font_size='11pt', text_align='left', text_color='#505050', text_baseline='top')
p.text(x=pd.to_datetime('1995-02-01'), y=71, text=['Women'], text_font_style='bold',
       text_font_size='11pt', text_align='left', text_color='#505050', text_baseline='top')
p.text(x=pd.to_datetime('1995-02-01'), y=81.8, text=['Total'], text_font_style='bold',
       text_font_size='11pt', text_align='left', text_color='#505050', text_baseline='top')
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/epop.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)